<a href="https://colab.research.google.com/github/pablOOO5/pablOOO5/blob/main/Clasificar_web_h2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapea y clasifica Web

=> Scrapea una web y la clasifica por tópicos. Muestra:

- Gráfica de tópicos de la web
- Gráfica de dispersión de los topicos
- Tabla con palabras por tópico
- Tabla de url por tópico con puntuación dentro del tópico.

---
1. Cambia en el código la url por la que quieras (ej. url = "https://www.debelareabogados.es/")
2. Pulsa el play
3. Espera a que el script acabe.
4. Si salen demasiados tópicos, en el último script selecciona el número de tópicos que quieres y dale al play en esa parte.

** **Nota**: La idea inicial de este colab parte de [un artículo de holistic seo](https://www.holisticseo.digital/python-seo/topic-modeling/) 

<br>

Un saludo desde Mallorca,

[Jose Gris](https://twitter.com/JoseGrisSEO) 😎
<br>

↓ El contenido aparecerá debajo del script ↓

<br>

---

Si te son útiles mis colabs [invítame a un café](https://www.buymeacoffee.com/josegris)

---

In [1]:
#------------------------------------------------------------------------------------
url = "https://credinfo.com.mx" #ej url = 'https://www.elpradopsicologos.es/'
#------------------------------------------------------------------------------------
!pip install advertools
import pandas as pd
from advertools import crawl

!rm -rf web.jl
crawl(url, 'web.jl', follow_links=True)

web = pd.read_json("web.jl", lines=True)
def rotulos(rotulo):
  print("\n\n")
  print("---------------------------------")
  print(rotulo)
  print("---------------------------------")

from google.colab import data_table
def pasarATabla(dataframe0, Nlineas=10):
  tabla = data_table.DataTable(dataframe0, include_index=False, num_rows_per_page=Nlineas)
  display(tabla)

def pasarListaATabla(lista,columnas):
  lista = pd.DataFrame (lista, columns = columnas )
  lista = data_table.DataTable(lista, include_index=True, num_rows_per_page=20)
  display(lista)

rotulos("Código de respuestas distinto de 200")
noCodigo200 = web[web['status'] != 200][["url", "title", "status"]]
pasarATabla(noCodigo200)

#Pillo lo que me interesa del scrapeado, y sólo códigos de respuesta 200
codigos200 = web[web['status'] == 200][["url","title", "h1", "h2"]]

#Me hago un corpus de datos con title y h2. Los h1 a menudo se acortan demasiado
interesantes = ["title", "h2"]
docs = []

for param in interesantes:

  espejo = codigos200[(codigos200[param].isna() ==False)][param].str.split("@@").explode().drop_duplicates().to_list()
  docs = docs + espejo
  
#Instalo spacy para quitar puntos, conjunciones, vertbos...
!pip install spacy
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp = es_core_news_sm.load()
import spacy

import re

#quito acentos para igualar las faltas
from unicodedata import normalize
def quitarAcentos(frase):
  frase = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", frase), 0, re.I
    )
  frase = normalize( 'NFC', frase)
  return frase

# Limpio el texto antes de procesarlo 
def limpiarTexto(texto):
  texto = re.sub(r'\s+', ' ', texto)
  texto = re.sub(r'\n+', ' ', texto)
  texto = re.sub(r'\t+', ' ', texto)
  texto = re.sub(r'http\S+', '', texto)
  texto = re.sub('[^a-zA-ZÀ-ÿ\u00f1\u00d1,.¿?¡!]', ' ', texto)
  doc = nlp(texto)
  evitar = {"DET", "CONJ", "CCONJ", "ADP", "ADV", "AUX", "PRON", "INTJ", "PUNCT", "SCONJ", "SYM", "VERB"} 
  soloPalabras = [token.text #lemma_
        for token in doc
        if not token.is_stop and not token.is_punct and token.pos_ not in evitar]
  texto = " ".join(soloPalabras)
  texto = texto.replace("  ", " ").strip()
  texto = texto.replace("(", "").replace(")", "")
  texto = quitarAcentos(texto).lower()
  return texto

docsLimpios = []
for doc in docs:
  docsLimpios.append(limpiarTexto(doc))

# Me instalo BERTOPIC
!pip install bertopic[gensim]
from bertopic import BERTopic

model = BERTopic(language="spanish") #, nr_topics="auto" reduciría topicos en auto

topics, probabilities = model.fit_transform(docsLimpios)
model.get_topic_info()

#Como info...
for param in interesantes:
  #identifico los que tienen el parametro nulo
  nulos = codigos200[(codigos200[param].isna())]
  rotulos(f"Sin {param}")
  pasarATabla(nulos)

  #identifico los que tienen el parametro duplicado
  duplicados = codigos200[(codigos200[param].isna()==False)]
  duplicados = duplicados[(codigos200.duplicated(param))]
  rotulos(f"{param} duplicado")
  pasarATabla(duplicados.head())

def recuperarTopico(doc):
  doc0 = str(doc) 
  try:
    #cat = similar_topic, similarity = model.find_topics(limpiarTexto(doc), top_n=1)
    #cat1 = model.get_topic_info(cat[0][0])
    #return cat1.Name.iloc[0] + "___"  + str(round(cat[1][0], 2))

    similar_topic, similarity = model.find_topics(limpiarTexto(doc), top_n=1)
    #topico = model.get_topic(similar_topic[0])
    topicos = model.get_topic_info()
    miTopico = topicos[topicos["Topic"] == similar_topic[0]]["Name"].iloc[0]
    return miTopico + "___"  + str(round(similarity[0], 2))

  except:
    return "None"

def graficas2():
  fig1 = model.visualize_barchart(n_words=6, top_n_topics=len(model.get_topics()))
  fig1.show()
  print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("- Hijos de las SERPs, yo, yo soy Jose Gris!")
  print("- Hiiiiiiiiiiiii, hiiiiiiiiiiiiiiii (relincho de caballos)")
  print("- No puede ser, Jose Gris mide más de dos metros y dicen que es capaz de posicionar webs en Flash")
  print("- ¡Vámonos, huyamos!")
  print("- Huid y vivireis, un tiempo al menos... Pero llegará un día, en vuestro lecho de muerte, en el que lo darías todo, TODO por una oportunidad,")
  print("      UNA SOLA OPORTUNIDAAAADD, DE VOLVER A LAS SERPSSSSS Y LUCHAR POR POSICIONAAAAAAAAARRRRRRRR!!!!!!!!!!!!!!!!!!!!!!!")
  print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  fig2 = model.visualize_topics()
  fig2.show()
  fig3  = model.visualize_hierarchy()
  fig3.show()
  
  #Creo tabla con definición de todos los tópicos
  todosLosTopicos = model.get_topics()
  topicos = []
  for topicoInt in todosLosTopicos:
    topico = [x[0] for x in todosLosTopicos[topicoInt]]
    topicos.append([topicoInt," - ".join(topico).strip().strip("-")])
  pasarListaATabla(topicos, ["Tópico", "Definición"])

  #creo tabla clasificando urls por tópico
  paramAComparar = "h1" #opcional clasificar por title
  titulos0 = codigos200[["url", paramAComparar]] 
  titulos0 = titulos0[titulos0[paramAComparar].isna() == False].drop_duplicates()
  titulos0["Categoria"] = titulos0.apply(lambda row: recuperarTopico(row[paramAComparar]), axis = 1)
  titulos0[['Categoria', 'Puntuación']] = titulos0['Categoria'].str.split("___", 1, expand=True)
  titulos0 = titulos0.sort_values(['Categoria', 'Puntuación'], ascending=[True, False])
  pasarATabla(titulos0, 20)

graficas2()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 21.8 MB/s 
     |████████████████████████████████| 271 kB 32.3 MB/s 
     |████████████████████████████████| 93 kB 3.3 MB/s 
     |████████████████████████████████| 3.1 MB 58.3 MB/s 
     |████████████████████████████████| 261 kB 47.7 MB/s 
     |████████████████████████████████| 4.0 MB 49.7 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=9998312aed0135d2b395cc6dcbbb7a75e1bb01a634052216f6bcf2e4c1039bd3
  Stored in directory: /root/.cache/pip/wheels/dc/b9/4a/948b1176e084b9e3f85e4ffc3d08f817b1fdf0d973bbb94f81
Successfully built PyDispatcher



---------------------------------
Código de respuestas distinto de 200
---------------------------------


,url,title,status
58,https://credinfo.com.mx/infonavit/infonatel/te...,Página no encontrada -,404
64,https://credinfo.com.mx/infonavit/infonatel/te...,Página no encontrada -,404
67,https://credinfo.com.mx/infonavit/cuanto-debo/...,Página no encontrada -,404
82,https://credinfo.com.mx/general/tarjeta-de-cre...,Página no encontrada -,404
132,https://credinfo.com.mx/precalificaciones.info...,Página no encontrada -,404


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 23:03:08.608217: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 27.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 9.3 MB/s 
     |████████████████████████████████| 85 kB 6.2 MB/s 
     |████████████████████████████████| 5.2 MB 58.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 662 kB 43.6 MB/s 
     |████████████████████████████████| 88 kB 9.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
     |████████████████████████████████| 5.8 MB 54.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.4 MB/s 
     |████████████████████████████████| 182 kB 103.0 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 
     |████████████████████████████████| 1.1 MB 52.3 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700877 sha256=8cc1ea22a1b0b10ae4ba2ebf96c294625f385d5dff

...




---------------------------------
h2 duplicado
---------------------------------


,url,title,h1,h2
105,https://credinfo.com.mx/infonavit/checar-saldo...,✔️ Checa Tu Saldo Mejoravit 2022 Gratis | Guía...,Aprende a checar tu saldo de Mejoravit y saber...,Mejoravit@@Cómo saber tu saldo de Mejoravit@@M...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
- Hijos de las SERPs, yo, yo soy Jose Gris!
- Hiiiiiiiiiiiii, hiiiiiiiiiiiiiiii (relincho de caballos)
- No puede ser, Jose Gris mide más de dos metros y dicen que es capaz de posicionar webs en Flash
- ¡Vámonos, huyamos!
- Huid y vivireis, un tiempo al menos... Pero llegará un día, en vuestro lecho de muerte, en el que lo darías todo, TODO por una oportunidad,
      UNA SOLA OPORTUNIDAAAADD, DE VOLVER A LAS SERPSSSSS Y LUCHAR POR POSICIONAAAAAAAAARRRRRRRR!!!!!!!!!!!!!!!!!!!!!!!
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Tópico,Definición
0,-1,gratis - puente - credito - buro - tarjeta - s...
1,0,coppel - credito - revolvente - buro - politic...
2,1,cambiavit - mejoravit - ciento - homoclave - d...
3,2,espacio - infonavit - situacion - encontrarlos...
4,3,infonavit - credito - paso - numero - use - te...
5,4,aportaciones - laboral - trabajador - historia...
6,5,casa - vivienda - hipotecario - banorte - aban...
7,6,asesor - publico - secretaria - hacienda - con...
8,7,ecotecnologias - hipoteca - verde - vale - pro...
9,8,deuda - fovissste - desapareces - mala - calif...


,url,h1,Categoria,Puntuación
45,https://credinfo.com.mx/general/credito-puente/,"Qué Es Un Crédito Puente. Función, Pros, Contr...",-1_gratis_puente_credito_buro,0.62
95,https://credinfo.com.mx/infonavit/puntos-credito/,Infonavit: ¿Cuántos Puntos Necesito Para Solic...,-1_gratis_puente_credito_buro,0.56
24,https://credinfo.com.mx/general/tarjeta-saldazo/,"Tarjeta Saldazo. Qué Es, Requisitos, Solicitud...",-1_gratis_puente_credito_buro,0.51
102,https://credinfo.com.mx/infonavit/solicitud-cr...,Cómo Llenar La Solicitud De Inscripción De Cré...,-1_gratis_puente_credito_buro,0.51
5,https://credinfo.com.mx/buro/,\nBuró,0_coppel_credito_revolvente_buro,0.76
...,...,...,...,...
55,https://credinfo.com.mx/general/linea-de-credito/,Cómo Aumentar Una Línea De Crédito. Paso A Paso,8_deuda_fovissste_desapareces_mala,0.42
10,https://credinfo.com.mx/infonavit/suspension-d...,Dónde Descargar Mi Aviso De Suspensión De Desc...,9_descuentos_aviso_descuento_salario,0.59
126,https://credinfo.com.mx/infonavit/aviso-retenc...,Infonavit: Cómo Obtengo Mi Aviso De Retención ...,9_descuentos_aviso_descuento_salario,0.57
78,https://credinfo.com.mx/infonavit/salario-base/,Salario Base De Cotización: Todo Lo Que Tienes...,9_descuentos_aviso_descuento_salario,0.56


↑ Los resultados aparecerán arriba => Puedes pulsar el botón de copiar en cada tabla para descargarlas

↓ Si quieres reducir el número de tópicos => cambia numTopicos al valor que quieras y pulsa el play en el código de abajo

In [ ]:
#Opción reducir el número de topicos
numTopicos = 15
new_topics, new_probs = model.reduce_topics(docsLimpios, topics, nr_topics=numTopicos)
graficas2()